In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re

# URL da página principal
url = "https://www.sicredi.com.br/home/"

# Função para extrair o conteúdo de uma página sem os textos dentro das tags <a> e <button>
def extrair_conteudo_sem_links_e_botoes(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Remove todas as tags <a> e <button> do conteúdo
            for tag in soup.find_all(['a', 'button']):
                tag.decompose()
            
            # Extrai o texto restante da página
            texto = soup.get_text(separator="\n", strip=True)
            
            # Remove slogans específicos
            texto = re.sub(r"\bcom quem contar\b", "", texto, flags=re.IGNORECASE)
            texto = re.sub(r"\bseja associado\b", "", texto, flags=re.IGNORECASE)
            
            # Remove linhas vazias e retorna o conteúdo limpo
            linhas = [linha.strip() for linha in texto.splitlines() if linha.strip()]
            return "\n".join(linhas)
        else:
            print(f"Erro ao acessar {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return None

# Função para coletar todos os links de uma página
def coletar_links(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            links = [a['href'] for a in soup.find_all('a', href=True)]
            links_completos = [link for link in links if link.startswith('http')]
            return list(set(links_completos))  # Remove URLs duplicadas
        else:
            print(f"Erro ao acessar {url}. Status code: {response.status_code}")
            return []
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return []

# Função principal para coletar dados de todas as páginas
def coletar_dados_pagina_principal(url):
    links = coletar_links(url)
    print(f"Encontrados {len(links)} links únicos na página principal.")

    conteudos_unicos = {}  # Dicionário para armazenar conteúdos únicos por URL
    for link in links:
        print(f"Acessando: {link}")
        conteudo = extrair_conteudo_sem_links_e_botoes(link)
        if conteudo and conteudo not in conteudos_unicos.values():  # Verifica se o conteúdo já existe
            conteudos_unicos[link] = conteudo
        time.sleep(2)

    # Salvar os dados em um arquivo JSON
    dados = [{'url': url, 'conteudo': conteudo} for url, conteudo in conteudos_unicos.items()]
    with open('conteudo_paginas_sem_links.json', 'w', encoding='utf-8') as file:
        json.dump(dados, file, ensure_ascii=False, indent=4)

    print("Dados tratados e salvos com sucesso!")

# Chama a função principal
coletar_dados_pagina_principal(url)


Encontrados 92 links únicos na página principal.
Acessando: https://www.sicredi.com.br/site/atendimento/canais/
Acessando: https://www.sicredi.com.br/site/credito/para-agronegocio/
Acessando: https://sicredi.cc/3PMnEAN
Acessando: https://www.sicredi.com.br/site/investimentos/analises-economicas-financeiras/
Acessando: https://www.sicredi.com.br/site/sobre-nos/noticias/institucional/sicredi-reconhecida-melhor-empresa-trabalhar-brasil/
Acessando: https://www.sicredi.com.br/site/cambio-e-comercio-exterior/
Acessando: https://www.sicredi.com.br/site/credito/para-empresa/
Acessando: https://www.sicredi.com.br/site/programa-juntos/
Acessando: https://www.sicredi.com.br/site/sobre-nos/relatorios/
Acessando: https://www.sicredi.com.br/sites/informativoconta/
Acessando: https://www.sicredi.com.br/site/seguros/para-agronegocio/
Acessando: https://sicredi.cc/44Ep3xT
Acessando: https://itunes.apple.com/br/app/sicredi-mobi-para-iphone/id1041468908
Acessando: https://www.sicredi.com.br/site/seguros/